# Import packages and data

In [119]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import pandas as pd
import numpy as np
import zipfile

%matplotlib inline

In [10]:
zf = zipfile.ZipFile('../data/taxi_trip_duration.zip') 
taxi_data = pd.read_csv(zf.open('train.csv'))

taxi_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


## Preprocessing

In [90]:
taxi_data['dropoff_datetime'] = pd.to_datetime(taxi_data['dropoff_datetime'])
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'])
taxi_data['month'] = taxi_data['dropoff_datetime'].dt.month

# HeatMap

In [91]:
# reference: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis

# After lots of samples, the average max number of points accepted by HeatMap is ~35.000 pairs (lat, lon)
MAX_NBR_RECORDS = 35000

map_init_coords = [taxi_data['pickup_latitude'].median(),taxi_data['pickup_longitude'].median()]

taxi_data_sample = taxi_data.sample(MAX_NBR_RECORDS, random_state=42)
heat_data = [[row['pickup_latitude'],row['pickup_longitude']] for index, row in taxi_data_sample.iterrows()]

m = folium.Map(map_init_coords)
HeatMap(heat_data).add_to(m)
m

In [138]:
heatmap_time = [[[row['pickup_latitude'], row['pickup_longitude']] 
                for index, row in taxi_data_sample.loc[taxi_data_sample['month'] == m].iterrows()]
                for m in taxi_data_sample['month'].drop_duplicates().sort_values()]

m = folium.Map(map_init_coords)
hm_t = HeatMapWithTime(heatmap_time,auto_play=True,max_opacity=0.8)
hm_t.add_to(m)

m